# Goal:

In this assignment, you will implement a [Classifier Free Guidance model](https://arxiv.org/pdf/2207.12598) class on MNIST dataset using PyTorch according to the guidence. The goal is to minimize the loss function and train the model to generate MNIST images with conditions on label.

The `Train` and `UNet` classes are already implemented for you. You need to implement the `CFGDiffusion` class (see details below). The images generated by the model will be automatically shown according to the `Trainer` class implementation. Make sure the generated images are shown in the output, it will be graded.

Grade:
- Explain why is the model called Classifier Free  and why Guidance (5 points).
- According to the paper, what would be an alternative of classifier free ? Explain how would the loss change in this alternative compared to the original DDPM loss ? (5 points)
- Implement CFGDiffusion class (20 points)
- Complete the Trainer.sample() method (10 points)
- Write a report to describe the sampled images generated by each epochs (5 points).

**Please note that the function to generate the images is already provided.**

---
Please DO NOT change the code provided, only add your own code where indicated. It is recommended that you **use CPU session to debug** when GPU is not necessary since Colab only gives 12 hrs of free GPU access at a time. If you use up the GPU resource, you may consider using Kaggle GPU resource. Thank you and good luck!

# Pre-determined config and given functions (no need to change)

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    pass 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Add the following files to your directory:
- args.py
- unet.py
- datasets.py
- utils.py

In [ ]:
import torch
import torch.utils.data
from torch import nn
import os
from typing import Tuple, Optional
import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.amp import GradScaler, autocast

from cfg_utils.args import *
from cfg_utils.dataset import *
from cfg_utils.unet import *


In [ ]:
print(f"Using {args.device} backend")

Using cuda backend


# Implement the Classifier Free Guidance Model 

To that end, refer to the training and sampling algorithms from the paper as well as the different equations. Less description is included here so that you're forced to learn how to refer to a paper. Still, note that guidences are also here to help you with what to fill in each function


In [2]:
# %%
import torch
import torch.utils.data
import torchvision
from torch import nn
from typing import Tuple, Optional
import torch.nn.functional as F
from tqdm import tqdm
from easydict import EasyDict
import matplotlib.pyplot as plt
from torch.amp import GradScaler, autocast
import os

from cfg_utils.args import *
from q1_train_vae import loss_function


class CFGDiffusion():
    def __init__(self, eps_model: nn.Module, n_steps: int, device: torch.device):
        super().__init__()
        self.eps_model = eps_model
        self.n_steps = n_steps

        self.lambda_min = -20
        self.lambda_max = 20

    ### UTILS
    def get_exp_ratio(self, l: torch.Tensor, l_prim: torch.Tensor):
        return torch.exp(l-l_prim)

    def get_lambda(self, t: torch.Tensor):
        u = t / self.n_steps

        device = t.device
        lambda_min = torch.tensor(self.lambda_min, device=device)
        lambda_max = torch.tensor(self.lambda_max, device=device)

        b = torch.atan(torch.exp(-lambda_max / 2))
        a = torch.atan(torch.exp(-lambda_min / 2)) - b

        lambda_t = -2 * torch.log(torch.tan(a * u + b))
        return lambda_t.view(-1, 1, 1, 1)

    def alpha_lambda(self, lambda_t: torch.Tensor):
        alpha_squared = 1 / (1 + torch.exp(-lambda_t))
        return torch.sqrt(alpha_squared)

    def sigma_lambda(self, lambda_t: torch.Tensor):
        alpha = self.alpha_lambda(lambda_t)
        sigma_squared = 1 - alpha ** 2
        return torch.sqrt(sigma_squared)

    ## Forward sampling
    def q_sample(self, x: torch.Tensor, lambda_t: torch.Tensor, noise: torch.Tensor):
        alpha = self.alpha_lambda(lambda_t)
        sigma = self.sigma_lambda(lambda_t)
        return alpha * x + sigma * noise

    def sigma_q(self, lambda_t: torch.Tensor, lambda_t_prim: torch.Tensor):
        sigma_lambda_sq = self.sigma_lambda(lambda_t) ** 2
        ratio = 1 - torch.exp(lambda_t - lambda_t_prim)
        var = ratio * sigma_lambda_sq
        return torch.sqrt(torch.clamp(var, min=1e-10))

    def sigma_q_x(self, lambda_t: torch.Tensor, lambda_t_prim: torch.Tensor):
        sigma_lambda_sq = self.sigma_lambda(lambda_t_prim) ** 2
        ratio = 1 - torch.exp(lambda_t - lambda_t_prim)
        var = ratio * sigma_lambda_sq
        return torch.sqrt(torch.clamp(var, min=1e-10))

    ### REVERSE SAMPLING
    def mu_p_theta(self, z_lambda_t: torch.Tensor, x: torch.Tensor, lambda_t: torch.Tensor, lambda_t_prim: torch.Tensor):
        exp_ratio = torch.exp(lambda_t - lambda_t_prim)
        alpha_t = self.alpha_lambda(lambda_t)
        alpha_t_prim = self.alpha_lambda(lambda_t_prim)

        term1 = exp_ratio * (alpha_t_prim / alpha_t) * z_lambda_t
        term2 = (1 - exp_ratio) * alpha_t_prim * x
        return term1 + term2

    def var_p_theta(self, lambda_t: torch.Tensor, lambda_t_prim: torch.Tensor, v: float=0.3):
        sigma_lambda_sq = self.sigma_lambda(lambda_t) ** 2
        sigma_lambda_prim_sq = self.sigma_lambda(lambda_t_prim) ** 2
        sigma_ratio = 1 - torch.exp(lambda_t - lambda_t_prim)

        base_var = sigma_ratio * sigma_lambda_sq
        base_var_prim = sigma_ratio * sigma_lambda_prim_sq

        # According to Eq (4): interpolate variance
        return torch.clamp((base_var_prim ** (1 - v)) * (base_var ** v), min=1e-10)

    def p_sample(self, z_lambda_t: torch.Tensor, lambda_t : torch.Tensor, lambda_t_prim: torch.Tensor,  x_t: torch.Tensor, set_seed=False):
        if set_seed:
            torch.manual_seed(42)

        mu = self.mu_p_theta(z_lambda_t, x_t, lambda_t, lambda_t_prim)
        var = self.var_p_theta(lambda_t, lambda_t_prim)

        noise = torch.randn_like(z_lambda_t)
        sample = mu + torch.sqrt(var) * noise

        return sample

    ### LOSS
    def loss(self, x0: torch.Tensor, labels: torch.Tensor, noise: Optional[torch.Tensor] = None, set_seed=False):
        if set_seed:
            torch.manual_seed(42)

        batch_size = x0.shape[0]
        dim = tuple(range(1, x0.ndim))

        # Step 1: Sample t ~ Uniform({0, ..., T-1})
        t = torch.randint(0, self.n_steps, (batch_size,), device=x0.device, dtype=torch.long)

        # Step 2: Get λ(t)
        lambda_t = self.get_lambda(t)  # (batch_size, 1, 1, 1)

        # Step 3: Sample noise ε ~ N(0, I)
        if noise is None:
            noise = torch.randn_like(x0)

        # Step 4: Compute z_λ = α(λ) * x0 + σ(λ) * ε
        z_lambda = self.q_sample(x0, lambda_t, noise)

        # Step 5: Predict noise using ε_θ(z_λ, labels)
        eps_pred = self.eps_model(z_lambda, labels)

        loss = (eps_pred - noise) ** 2  # (batch_size, C, H, W)
        loss = loss.sum(dim=dim).mean()  # sum over pixels, mean over batch

        return loss

# Finish implementation of the Trainer.sample() method

In [ ]:
import copy
import numpy as np
from q3_trainer_cfg import *

class Trainer:
    def __init__(self, args, eps_model, diffusion_model):

        self.eps_model = eps_model.to(args.device)

        self.diffusion = diffusion_model

        self.optimizer = torch.optim.Adam(
            self.eps_model.parameters(), lr=args.learning_rate
        )
        self.args = args
        self.current_epoch = 0

        self.ema = EMA(0.995)
        self.ema_model = copy.deepcopy(self.eps_model).eval().requires_grad_(False)


    def train_epoch(self, dataloader, scaler):
        current_lr = round(self.optimizer.param_groups[0]['lr'], 8)
        i = 0
        running_loss = 0.
        with tqdm(range(len(dataloader)), desc=f'Epoch : - lr: - Loss :') as progress:
            for x0, labels in dataloader:
                i += 1
                # Move data to device
                x0 = x0.to(self.args.device)
                # Use guidance
                labels = labels.to(self.args.device)
                if np.random.random() < 0.1:
                    labels = None

                # Calculate the loss
                with autocast(device_type=self.args.device, enabled=self.args.fp16_precision):
                    loss = self.diffusion.loss(x0, labels)

                # Zero gradients
                self.optimizer.zero_grad()
                # Backward pass
                scaler.scale(loss).backward()
                scaler.step(self.optimizer)
                scaler.update()
                # loss.backward()
                # self.optimizer.step()
                self.ema.step_ema(self.ema_model, self.eps_model)

                running_loss += loss.item()

                self.loss_per_iter.append(running_loss / i)
                progress.update()
                progress.set_description(f'Epoch: {self.current_epoch}/{self.args.epochs} - lr: {current_lr} - Loss: {round(running_loss / i, 2)}')
            progress.set_description(f'Epoch: {self.current_epoch}/{self.args.epochs} - lr: {current_lr} - Loss: {round(running_loss / len(dataloader), 2)}')

            # Step the scheduler after each epoch
            self.scheduler.step()

    def train(self, dataloader):
            scaler = GradScaler(device=self.args.device, enabled=self.args.fp16_precision)
            self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=5, gamma=0.5)
            start_epoch = self.current_epoch
            self.loss_per_iter = []
            for current_epoch in range(start_epoch, self.args.epochs):
                self.current_epoch = current_epoch
                self.train_epoch(dataloader, scaler)
                if current_epoch % self.args.show_every_n_epochs == 0:
                    self.sample(cfg_scale=self.args.cfg_scale)

                if (current_epoch + 1) % self.args.save_every_n_epochs == 0:
                    self.save_model()

    def sample(self, labels=None, cfg_scale=3., n_steps=None, set_seed=False):
        if set_seed:
            torch.manual_seed(42)
        if n_steps is None:
            n_steps = self.args.n_steps

        self.eps_model.eval()
        with torch.no_grad():
            z_t = torch.randn(
                    [
                        self.args.n_samples,
                        self.args.image_channels,
                        self.args.image_size,
                        self.args.image_size,
                    ],
                    device=self.args.device,
                )

            if labels == None:
                labels = torch.randint(0, 9, (self.args.n_samples,), device=self.args.device)

            if self.args.nb_save is not None:
                saving_steps = [n_steps - 1]

            # Remove noise for $T$ steps
            for t_ in tqdm(range(n_steps)):
                t_val = n_steps - 1 - t_
                t = torch.full((self.args.n_samples,), t_val, device=z_t.device, dtype=torch.long)
                # t_prim = torch.full((self.args.n_samples,), t_val+1, device=z_t.device, dtype=torch.long)

                lambda_t = self.diffusion.get_lambda(t)
                lambda_t_prim = self.diffusion.get_lambda(t - 1)

                eps_cond = self.eps_model(z_t, labels)
                eps_uncond = self.eps_model(z_t, None)
                eps_theta = (1 + cfg_scale) * eps_cond - cfg_scale * eps_uncond

                alpha_lambda = self.diffusion.alpha_lambda(lambda_t)
                sigma_lambda = self.diffusion.sigma_lambda(lambda_t)
                x_t = (z_t - sigma_lambda * eps_theta) / alpha_lambda

                z_t = self.diffusion.p_sample(z_t, lambda_t, lambda_t_prim, x_t)

                if self.args.nb_save is not None and t_ in saving_steps:
                    print(f"Showing/saving samples from epoch {self.current_epoch} with labels: {labels.tolist()}")
                    self.show_save(
                        img_tensor=x_t,
                        labels=labels,
                        show=True,
                        save=True,
                        file_name=f"CFG_epoch_{self.current_epoch}_sample_{t_}.png"
                    )

            self.eps_model.train()
        return x_t

    def save_model(self):
        torch.save({
                'epoch': self.current_epoch,
                'model_state_dict': self.eps_model.state_dict(),
                'optimizer_state_dict': self.optimizer.state_dict(),
                }, self.args.MODEL_PATH)

    def show_save(self, img_tensor, labels=None, show=True, save=True, file_name="sample.png"):
        fig, axs = plt.subplots(3, 3, figsize=(10, 10))  # Create a 4x4 grid of subplots
        assert img_tensor.shape[0] >= 9, "Number of images should be at least 9"
        img_tensor = img_tensor[:9]
        for i, ax in enumerate(axs.flat):
            # Remove the channel dimension and convert to numpy
            img = img_tensor[i].squeeze().cpu().numpy()
            label = labels[i].item()
            ax.imshow(img, cmap="gray")  # Display the image in grayscale
            ax.set_title(f'Digit:{label}')
            ax.axis("off")  # Hide the axis

        plt.tight_layout()
        if save:
            os.makedirs("images", exist_ok=True)
            plt.savefig(os.path.join("images", file_name))
        if show:
            plt.show()
        plt.close(fig)

In [ ]:
# %%

dataloader = torch.utils.data.DataLoader(
    MNISTDataset(),
    batch_size=args.batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=0,
    pin_memory=True,
)

eps_model = UNet_conditional(c_in=1, c_out=1, num_classes=10)

diffusion_model = CFGDiffusion(
            eps_model=eps_model,
            n_steps=args.n_steps,
            device=args.device,
        )

trainer = Trainer(args, eps_model, diffusion_model)

trainer.train(dataloader)